In [1]:
from inference import inference
from architecture import nova


importlib.reload(nova)
importlib.reload(model_io)

2024-12-16 11:41:48.487133: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


FileNotFoundError: [Errno 2] No such file or directory: '/vocabulary.txt'

## Inference Testing

In [1]:
test_string = "the cow jumped over the moon"

inference.Generator(

## Model Testing

In [2]:
test_string = [["the", "cow"], ["hello", "world"]]

model = nova.Model()

In [3]:
run = model.fPass(test_string)

run

Adding a new word to the model...
Done.
Retrieving existing embedding
Adding a new word to the model...
Retrieving existing embedding
Adding a new word to the model...
Retrieving existing embedding
Adding a new word to the model...
Retrieving existing embedding


<tf.Tensor: shape=(2,), dtype=int64, numpy=array([38, 38])>

In [4]:
model.get_config()

{'d_model': 256, 'num_heads': 4, 'dff': 1024, 'vocab_len': 91, 'num_tfmrs': 32}

In [5]:
model_io.save(model = model, save_dir = "/Users/joericks/Desktop/nova/model")

Model Saved Successfully


/usr/local/lib/python3.9/site-packages/keras/src/saving/saving_api.py:102: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  return saving_lib.save_model(model, filepath)


In [11]:
loaded_model = model_io.load(save_dir = "/Users/joericks/Desktop/nova/model")

loaded_model.fPass(test_string)

Adding a new word to the model...
Done.
Retrieving existing embedding
Adding a new word to the model...
Retrieving existing embedding
Adding a new word to the model...
Retrieving existing embedding
Adding a new word to the model...
Retrieving existing embedding


<tf.Tensor: shape=(2,), dtype=int64, numpy=array([5, 5])>